# Практическое задание к уроку 8. Рекуррентные нейронные сети RNN LSTM GRU


Данные берем отызывы за лето

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи 

1. построить свёрточные архитектуры 

2. построить различные архитектуры с RNN

3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [1]:
# # загрузка файла с локального диска
# from google.colab import files
# uploaded = files.upload()

In [2]:
!pip install stop_words

In [3]:
!pip install pymorphy2

In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [5]:
!pip install utils

In [6]:
!pip install daal==2021.4.0

  Using cached daal-2021.4.0-py2.py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.whl (189.9 MB)
  Using cached tbb-2021.6.0-py2.py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.whl (1.0 MB)
  Attempting uninstall: tbb
    Found existing installation: TBB 0.2
ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [7]:
!pip install typing-extensions>=3.10.0

zsh:1: 3.10.0 not found


In [8]:
# !pip install --upgrade numpy==1.18.5
# !pip install --upgrade numpy==1.19.2
# !pip install --upgrade numpy==1.18.5
!pip install --upgrade numpy==1.21.0

In [9]:
!pip install --upgrade pandas==1.2.0

In [10]:
!pip install keras==2.4.3

In [11]:
!pip install tensorflow==2.5.0

  Using cached numpy-1.19.5-cp39-cp39-macosx_10_9_x86_64.whl (15.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.0
    Uninstalling numpy-1.21.0:
      Successfully uninstalled numpy-1.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
bokeh 2.4.2 requires typing-extensions>=3.10.0, but you have typing-extensions 3.7.4.3 which is incompatible.


In [12]:
!pip install dca

In [13]:
import dca
import pkg_resources

import pandas as pd
import numpy as np
import re

pd.set_option('max_colwidth', 120)

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

# import keras
# from keras.models import Sequential, Model
# from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.callbacks import TensorBoard 
# from keras.objectives import categorical_crossentropy
# from keras.callbacks import EarlyStopping


import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, SpatialDropout1D, concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import SimpleRNN, GRU, Masking
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, Flatten, MaxPooling1D
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.preprocessing.text import Tokenizer

#### Обработка данных

In [14]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_val = pd.read_csv("val.csv")

In [15]:
df_train.head(5)

,id,text,class
0,0,"@alisachachka не уезжаааааааай. :(❤ я тоже не хочу, чтобы ты уезжала.",0
1,1,"RT @GalyginVadim: Ребята и девчата!\nВсе в кино!!! ""Вот Это Любовь!""\nСегодня! Завтра! И потом!)))))\n#вотэтолюбовь",1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретвит((((( RT,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Запретный плод. :),1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса не боится мороза и .......)),1


In [16]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [17]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [18]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [19]:
y_train = df_train['class'].values
y_val = df_val['class'].values

#### Сверточная сеть

In [20]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    output_dim=128, 
                    input_length=training_length, 
                    trainable=True,
                    mask_zero=True)
         )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )

In [21]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 64s 199ms/step - loss: 0.5398 - accuracy: 0.7165 - val_loss: 0.4800 - val_accuracy: 0.7596


In [22]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 8ms/step - loss: 0.5086 - accuracy: 0.7446


Test score: 0.5086029171943665
Test accuracy: 0.7445664405822754


In [23]:
compare_result = pd.DataFrame(columns=['Model', 'Score', 'Accuracy'])
compare_result.loc[0] = ['CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.508603,0.744566


#### RNN

Простая рекуррентная сеть

In [24]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
         )
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )

In [25]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 23s 70ms/step - loss: 0.5611 - accuracy: 0.6962 - val_loss: 0.4941 - val_accuracy: 0.7546


In [26]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 10ms/step - loss: 0.5085 - accuracy: 0.7414


Test score: 0.5084782242774963
Test accuracy: 0.7414363026618958


In [27]:
compare_result.loc[1] = ['RNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.508603,0.744566
1,RNN,0.508478,0.741436


#### LSTM

In [28]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy']
              )

In [29]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train,
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 40s 119ms/step - loss: 0.5552 - accuracy: 0.7069 - val_loss: 0.4929 - val_accuracy: 0.7538


In [30]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 33ms/step - loss: 0.5131 - accuracy: 0.7418


Test score: 0.5130863785743713
Test accuracy: 0.7417889833450317


In [31]:
compare_result.loc[2] = ['LSTM', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.508603,0.744566
1,RNN,0.508478,0.741436
2,LSTM,0.513086,0.741789


#### GRU

In [32]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    trainable=True,
                    mask_zero=True)
         )
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
              )

In [33]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train,batch_size=512,epochs=10,verbose=1,validation_split=0.1,callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 31s 93ms/step - loss: 0.5513 - accuracy: 0.7087 - val_loss: 0.4949 - val_accuracy: 0.7567


In [34]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 23ms/step - loss: 0.5221 - accuracy: 0.7410


Test score: 0.5221145153045654
Test accuracy: 0.7409513592720032


In [35]:
compare_result.loc[3] = ['GRU', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.508603,0.744566
1,RNN,0.508478,0.741436
2,LSTM,0.513086,0.741789
3,GRU,0.522115,0.740951


In [36]:
compare_result

,Model,Score,Accuracy
0,CNN,0.508603,0.744566
1,RNN,0.508478,0.741436
2,LSTM,0.513086,0.741789
3,GRU,0.522115,0.740951


#### Совместные архитектуры CNN -> RNN

##### CNN-SimpleRNN

In [37]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 24s 74ms/step - loss: 0.5590 - accuracy: 0.7002 - val_loss: 0.4938 - val_accuracy: 0.7578


In [38]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 15ms/step - loss: 0.5027 - accuracy: 0.7451


Test score: 0.5027254819869995
Test accuracy: 0.7451395392417908


In [39]:
compare_result.loc[4] = ['CNN-SimpleRNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.508603,0.744566
1,RNN,0.508478,0.741436
2,LSTM,0.513086,0.741789
3,GRU,0.522115,0.740951
4,CNN-SimpleRNN,0.502725,0.745140


##### CNN - LSTM

In [40]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train,
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 43s 132ms/step - loss: 0.5513 - accuracy: 0.7085 - val_loss: 0.4869 - val_accuracy: 0.7600


In [41]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 21ms/step - loss: 0.5024 - accuracy: 0.7484


Test score: 0.5023513436317444
Test accuracy: 0.7483577728271484


In [42]:
compare_result.loc[5] = ['CNN - LSTM', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.508603,0.744566
1,RNN,0.508478,0.741436
2,LSTM,0.513086,0.741789
3,GRU,0.522115,0.740951
4,CNN-SimpleRNN,0.502725,0.745140
5,CNN - LSTM,0.502351,0.748358


##### CNN-GRU

In [43]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 38s 115ms/step - loss: 0.5506 - accuracy: 0.7070 - val_loss: 0.4875 - val_accuracy: 0.7585


In [44]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 27ms/step - loss: 0.5301 - accuracy: 0.7374


Test score: 0.5300748944282532
Test accuracy: 0.7373804450035095


In [45]:
compare_result.loc[6] = ['CNN-GRU', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.508603,0.744566
1,RNN,0.508478,0.741436
2,LSTM,0.513086,0.741789
3,GRU,0.522115,0.740951
4,CNN-SimpleRNN,0.502725,0.745140
5,CNN - LSTM,0.502351,0.748358
6,CNN-GRU,0.530075,0.737380


#### Совместные архитектуры RNN -> CNN

##### SimpleRNN-CNN

In [46]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64, return_sequences=True))
model.add(Conv1D(64, 3))
model.add(Activation("relu"))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 31s 95ms/step - loss: 0.6711 - accuracy: 0.5471 - val_loss: 0.6583 - val_accuracy: 0.5621


In [47]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 22ms/step - loss: 0.6670 - accuracy: 0.5501


Test score: 0.6670083403587341
Test accuracy: 0.5501370429992676


In [48]:
compare_result.loc[7] = ['SimpleRNN-CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.508603,0.744566
1,RNN,0.508478,0.741436
2,LSTM,0.513086,0.741789
3,GRU,0.522115,0.740951
4,CNN-SimpleRNN,0.502725,0.745140
5,CNN - LSTM,0.502351,0.748358
6,CNN-GRU,0.530075,0.737380
7,SimpleRNN-CNN,0.667008,0.550137


##### LSTM - CNN

In [49]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30)
          )
model.add(LSTM(64, recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train,
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 44s 135ms/step - loss: 0.6736 - accuracy: 0.5393 - val_loss: 0.6607 - val_accuracy: 0.5557


In [50]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 43ms/step - loss: 0.6648 - accuracy: 0.5460


Test score: 0.6647577285766602
Test accuracy: 0.5460318326950073


In [51]:
compare_result.loc[8] = ['LSTM - CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.508603,0.744566
1,RNN,0.508478,0.741436
2,LSTM,0.513086,0.741789
3,GRU,0.522115,0.740951
4,CNN-SimpleRNN,0.502725,0.745140
5,CNN - LSTM,0.502351,0.748358
6,CNN-GRU,0.530075,0.737380
7,SimpleRNN-CNN,0.667008,0.550137
8,LSTM - CNN,0.664758,0.546032


##### GRU-CNN

In [52]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 44s 134ms/step - loss: 0.6732 - accuracy: 0.5412 - val_loss: 0.6623 - val_accuracy: 0.5466


In [53]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 42ms/step - loss: 0.6655 - accuracy: 0.5448


Test score: 0.6654859185218811
Test accuracy: 0.5447692275047302


In [54]:
compare_result.loc[9] = ['GRU - CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.508603,0.744566
1,RNN,0.508478,0.741436
2,LSTM,0.513086,0.741789
3,GRU,0.522115,0.740951
4,CNN-SimpleRNN,0.502725,0.745140
5,CNN - LSTM,0.502351,0.748358
6,CNN-GRU,0.530075,0.737380
7,SimpleRNN-CNN,0.667008,0.550137
8,LSTM - CNN,0.664758,0.546032
9,GRU - CNN,0.665486,0.544769


***Вывод:*** кроме RNN -> CNN модели показывают примерно одинаковую результативность.